In [26]:
#
# Se crea el directorio de entrada
#
!rm -rf /tmp/wordcount
!mkdir -p /tmp/wordcount/input
%cd /tmp/wordcount
!ls

sh: 0: getcwd() failed: No such file or directory
/tmp/wordcount
input


In [27]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [28]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [29]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing input/text2.txt


In [30]:
#
# Se usan un directorio temporal en el HDFS. La siguiente
# instrucción muestra el contenido del dicho directorio
#
!hdfs dfs -ls /tmp

Found 3 items
-rw-r--r--   1 root supergroup      88756 2022-11-21 16:45 /tmp/diabetes.csv
-rw-r--r--   1 root supergroup    9719482 2022-11-21 17:02 /tmp/flights.csv
drwxr-xr-x   - root supergroup          0 2022-11-21 17:13 /tmp/flightsWithDelays.orc


In [31]:
#
# Crea la carpeta wordcount en el hdfs
#
!hdfs dfs -mkdir /tmp/wordcount

In [32]:
#
# Verifica la creación de la carpeta
#
!hdfs dfs -ls /tmp/

Found 4 items
-rw-r--r--   1 root supergroup      88756 2022-11-21 16:45 /tmp/diabetes.csv
-rw-r--r--   1 root supergroup    9719482 2022-11-21 17:02 /tmp/flights.csv
drwxr-xr-x   - root supergroup          0 2022-11-21 17:13 /tmp/flightsWithDelays.orc
drwxr-xr-x   - root supergroup          0 2022-11-21 17:22 /tmp/wordcount


In [33]:
#
# Copia los archvios del directorio local wordcount/
# al directorio /tmp/wordcount/ en el hdfs
#
!hdfs dfs -copyFromLocal input  /tmp/wordcount/

In [34]:
#
# Verifica que los archivos esten copiados en el hdfs
#
!hdfs dfs -ls /tmp/wordcount
!hdfs dfs -ls /tmp/wordcount/input

Found 1 items
drwxr-xr-x   - root supergroup          0 2022-11-21 17:22 /tmp/wordcount/input
Found 3 items
-rw-r--r--   1 root supergroup       1082 2022-11-21 17:22 /tmp/wordcount/input/text0.txt
-rw-r--r--   1 root supergroup        349 2022-11-21 17:22 /tmp/wordcount/input/text1.txt
-rw-r--r--   1 root supergroup        435 2022-11-21 17:22 /tmp/wordcount/input/text2.txt


In [35]:
#
# findspark: Permite usar PySpark como una libreria de Python
#
import findspark
findspark.init()

#
# Importa las librerias requeridas para conectar
# a Python con PySpark
#
from pyspark import SparkConf, SparkContext

#
# operador de agregación (MapReduce)
#
from operator import add

#
# Nombre de la aplicación en el cluster
#
APP_NAME = "My Spark Application"

#
# Configure Spark
#
conf = SparkConf().setAppName(APP_NAME)
sc.stop()
sc = SparkContext(conf=conf)
sc.setLogLevel('WARN')

In [37]:
#
# Lee los archivos del HDFS y los carga a la variable text
#
text = sc.textFile("/tmp/wordcount/input/*.txt")

# Se imprimen las primeras 10 líneas
text.collect()[0:10]

['Analytics is the discovery, interpretation, and communication of meaningful patterns',
 'in data. Especially valuable in areas rich with recorded information, analytics relies',
 'on the simultaneous application of statistics, computer programming and operations research',
 'to quantify performance.',
 '',
 'Organizations may apply analytics to business data to describe, predict, and improve business',
 'performance. Specifically, areas within analytics include predictive analytics, prescriptive',
 'analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big',
 'Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,',
 'marketing optimization and marketing mix modeling, web analytics, call analytics, speech']

In [38]:
#
# separa por palabras (split) con una palabra por registro
#
words = text.flatMap(lambda x: x.split())

# Se imprimen las primeras 10 palabras
words.collect()[0:10]

['Analytics',
 'is',
 'the',
 'discovery,',
 'interpretation,',
 'and',
 'communication',
 'of',
 'meaningful',
 'patterns']

In [40]:
#
# Genera las parejas <clave, valor> representandolas
# com la tupla (word, 1)
#
wc = words.map(lambda x: (x,1))
wc.collect()[0:10]

[('Analytics', 1),
 ('is', 1),
 ('the', 1),
 ('discovery,', 1),
 ('interpretation,', 1),
 ('and', 1),
 ('communication', 1),
 ('of', 1),
 ('meaningful', 1),
 ('patterns', 1)]

In [41]:
#
# Suma los valores para la misma clave. Spark internamente ordena por claves
#
counts = wc.reduceByKey(add)
counts.collect()[0:10]

[('interpretation,', 1),
 ('of', 8),
 ('in', 5),
 ('data.', 1),
 ('Especially', 1),
 ('analytics', 8),
 ('simultaneous', 1),
 ('operations', 1),
 ('research', 2),
 ('quantify', 1)]

In [42]:
#
# Escribe los resultados al directorio `/tmp/output`
#
counts.saveAsTextFile("/tmp/wordcount/output")

In [43]:
!hdfs dfs -ls /tmp/wordcount/

Found 2 items
drwxr-xr-x   - root supergroup          0 2022-11-21 17:22 /tmp/wordcount/input
drwxr-xr-x   - root supergroup          0 2022-11-21 17:23 /tmp/wordcount/output


In [44]:
#
# Archivos con los resultados. Note que se
# generan varios archivos de resultados.
#
!hdfs dfs -ls /tmp/wordcount/output/

Found 5 items
-rw-r--r--   1 root supergroup          0 2022-11-21 17:23 /tmp/wordcount/output/_SUCCESS
-rw-r--r--   1 root supergroup        778 2022-11-21 17:23 /tmp/wordcount/output/part-00000
-rw-r--r--   1 root supergroup        562 2022-11-21 17:23 /tmp/wordcount/output/part-00001
-rw-r--r--   1 root supergroup        510 2022-11-21 17:23 /tmp/wordcount/output/part-00002
-rw-r--r--   1 root supergroup        594 2022-11-21 17:23 /tmp/wordcount/output/part-00003


In [45]:
!hdfs dfs -cat /tmp/wordcount/output/part-00000

('interpretation,', 1)
('of', 8)
('in', 5)
('data.', 1)
('Especially', 1)
('analytics', 8)
('simultaneous', 1)
('operations', 1)
('research', 2)
('quantify', 1)
('Organizations', 1)
('may', 1)
('business', 4)
('predict,', 1)
('include', 1)
('decision', 1)
('descriptive', 1)
('store', 1)
('optimization,', 2)
('modeling,', 2)
('speech', 1)
('promotion', 1)
('risk', 1)
('fraud', 1)
('Since', 1)
('algorithms', 1)
('used', 3)
('harness', 1)
('current', 1)
('field', 1)
('involves', 1)
('studying', 1)
('potential', 1)
('trends,', 1)
('performance', 1)
('goal', 1)
('changes.', 1)
('process', 1)
('draw', 1)
('specialized', 1)
('systems', 1)
('software.', 1)
('techniques', 1)
('are', 1)
('commercial', 1)
('organizations', 1)
('disprove', 1)
('scientific', 1)
('hypotheses.', 1)


In [46]:
#
# Copia los archivos de resultados a la maquina local
#
!mkdir -p output
!hdfs dfs -getmerge /tmp/wordcount/output/* output/results.txt

In [47]:
!head output/results.txt

('interpretation,', 1)
('of', 8)
('in', 5)
('data.', 1)
('Especially', 1)
('analytics', 8)
('simultaneous', 1)
('operations', 1)
('research', 2)
('quantify', 1)


In [21]:
#
# Limpieza
#
!rm -rf input
!rm -rf output
!hdfs dfs -rm -r -f /tmp/wordcount/

Deleted /tmp/wordcount
